# Tóm tắt văn bản với Claude trong LangChain

## Giới thiệu

Trong bài học này, chúng ta sẽ học cách sử dụng Claude để tóm tắt văn bản dài một cách hiệu quả. LangChain cung cấp nhiều phương pháp tóm tắt khác nhau để xử lý các loại tài liệu khác nhau.

### Các phương pháp tóm tắt chính:

1. **Stuff**: Đưa toàn bộ văn bản vào một prompt duy nhất
2. **Map-Reduce**: Tóm tắt từng phần riêng biệt, sau đó kết hợp các tóm tắt
3. **Refine**: Tóm tắt lần lượt từng phần và cải thiện dần

### Khi nào sử dụng phương pháp nào?

- **Stuff**: Văn bản ngắn, vừa với context window
- **Map-Reduce**: Văn bản rất dài, cần xử lý song song
- **Refine**: Cần tóm tắt chi tiết và chính xác nhất

## Cài đặt và Import

In [ ]:
# Cài đặt các thư viện cần thiết
# pip install langchain langchain-anthropic langchain-community python-dotenv

In [ ]:
import os
from dotenv import load_dotenv

# Import LangChain components
from langchain_anthropic import ChatAnthropic
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate

# Load environment variables
load_dotenv()

# Khởi tạo Claude
llm = ChatAnthropic(
    model="claude-3-5-sonnet-20241022",
    temperature=0.3,
    api_key=os.getenv("ANTHROPIC_API_KEY")
)

print("✅ Đã khởi tạo Claude thành công!")

## Chuẩn bị dữ liệu mẫu

Chúng ta sẽ tạo một văn bản dài để thực hành các phương pháp tóm tắt khác nhau.

In [ ]:
# Tạo văn bản mẫu dài về trí tuệ nhân tạo
long_text = """
Trí tuệ nhân tạo (AI) đã trở thành một trong những công nghệ quan trọng nhất của thế kỷ 21. 
Từ những ngày đầu của khoa học máy tính, con người đã mơ ước tạo ra những cỗ máy có thể suy nghĩ và học hỏi như con người.

Lịch sử phát triển của AI có thể chia thành nhiều giai đoạn. Giai đoạn đầu (1950-1970) được đánh dấu bởi sự ra đời của những chương trình AI đầu tiên.
Alan Turing đã đề xuất "Turing Test" để đánh giá khả năng thông minh của máy tính. Trong giai đoạn này, các nhà nghiên cứu tập trung vào việc giải quyết các bài toán logic và toán học.

Giai đoạn thứ hai (1970-1990) chứng kiến sự phát triển của các hệ thống chuyên gia. Những hệ thống này được thiết kế để mô phỏng khả năng ra quyết định của các chuyên gia trong các lĩnh vực cụ thể.
Mặc dù có những thành công nhất định, nhưng các hệ thống chuyên gia gặp phải nhiều hạn chế do không thể xử lý được những tình huống mới và phức tạp.

Giai đoạn thứ ba (1990-2010) đánh dấu sự trỗi dậy của machine learning. Thay vì lập trình cứng nhắc, các thuật toán machine learning cho phép máy tính học từ dữ liệu.
Các kỹ thuật như neural networks, decision trees, và support vector machines được phát triển và ứng dụng rộng rãi.

Giai đoạn hiện tại (2010-nay) được gọi là kỷ nguyên của deep learning. Sự phát triển mạnh mệnh của deep neural networks đã tạo ra những đột phá quan trọng trong nhiều lĩnh vực.
Các mô hình như GPT, BERT, và Claude đã chứng minh khả năng xử lý ngôn ngữ tự nhiên ở mức độ gần như con người.

Ứng dụng của AI trong đời sống hiện đại rất đa dạng. Trong y tế, AI giúp chẩn đoán bệnh, phát triển thuốc mới, và cá nhân hóa điều trị.
Trong giao thông, xe tự lái đang dần trở thành hiện thực. Trong kinh doanh, AI được sử dụng để phân tích dữ liệu, dự đoán thị trường, và tự động hóa quy trình.

Tuy nhiên, sự phát triển của AI cũng đặt ra nhiều thách thức. Vấn đề về đạo đức AI, tính minh bạch của thuật toán, và tác động đến việc làm là những quan ngại chính.
Cần có những quy định và tiêu chuẩn để đảm bảo AI phát triển theo hướng có lợi cho nhân loại.

Tương lai của AI hứa hẹn sẽ mang lại những thay đổi sâu sắc hơn nữa. Artificial General Intelligence (AGI) - AI có khả năng suy nghĩ và học hỏi như con người ở mọi lĩnh vực - có thể sẽ trở thành hiện thực trong vài thập kỷ tới.
Điều này sẽ mở ra những cơ hội to lớn nhưng cũng đòi hỏi sự chuẩn bị kỹ lưỡng từ xã hội.

Kết luận, trí tuệ nhân tạo đã và đang thay đổi thế giới theo những cách mà chúng ta chưa từng tưởng tượng. 
Việc hiểu và sử dụng AI một cách có trách nhiệm sẽ là chìa khóa để tận dụng tối đa những lợi ích mà công nghệ này mang lại.
"""

print(f"Độ dài văn bản: {len(long_text)} ký tự")
print(f"Số từ ước tính: {len(long_text.split())} từ")

## 1. Phương pháp Stuff

Đây là phương pháp đơn giản nhất - đưa toàn bộ văn bản vào một prompt duy nhất để tóm tắt.

In [ ]:
# Tạo document từ văn bản
docs = [Document(page_content=long_text)]

# Tạo summarization chain với phương pháp "stuff"
stuff_chain = load_summarize_chain(
    llm=llm,
    chain_type="stuff",
    verbose=True
)

print("🔄 Đang tóm tắt bằng phương pháp Stuff...")
stuff_summary = stuff_chain.run(docs)

print("\n📝 KẾT QUẢ TÓM TẮT (STUFF):")
print("=" * 50)
print(stuff_summary)

### Tùy chỉnh prompt cho phương pháp Stuff

In [ ]:
# Tạo custom prompt cho tóm tắt
custom_prompt = PromptTemplate(
    template="""
Hãy tóm tắt văn bản sau đây thành một đoạn ngắn gọn, tập trung vào:
- Các ý chính và quan trọng nhất
- Thông tin có giá trị thực tiễn
- Kết luận hoặc thông điệp chính

Văn bản cần tóm tắt:
{text}

TÓM TẮT:""",
    input_variables=["text"]
)

# Tạo chain với custom prompt
custom_stuff_chain = load_summarize_chain(
    llm=llm,
    chain_type="stuff",
    prompt=custom_prompt,
    verbose=True
)

print("🔄 Đang tóm tắt bằng custom prompt...")
custom_stuff_summary = custom_stuff_chain.run(docs)

print("\n📝 KẾT QUẢ TÓM TẮT (CUSTOM STUFF):")
print("=" * 50)
print(custom_stuff_summary)

## 2. Phương pháp Map-Reduce

Phương pháp này chia văn bản thành nhiều phần nhỏ, tóm tắt từng phần riêng biệt (Map), sau đó kết hợp các tóm tắt thành một tóm tắt cuối cùng (Reduce).

In [ ]:
# Chia văn bản thành các chunks nhỏ hơn
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
    separators=["\n\n", "\n", ".", " "]
)

# Tạo documents từ các chunks
split_docs = text_splitter.create_documents([long_text])

print(f"Văn bản đã được chia thành {len(split_docs)} phần")
for i, doc in enumerate(split_docs):
    print(f"Phần {i+1}: {len(doc.page_content)} ký tự")

In [ ]:
# Tạo Map-Reduce chain
map_reduce_chain = load_summarize_chain(
    llm=llm,
    chain_type="map_reduce",
    verbose=True
)

print("🔄 Đang tóm tắt bằng phương pháp Map-Reduce...")
map_reduce_summary = map_reduce_chain.run(split_docs)

print("\n📝 KẾT QUẢ TÓM TẮT (MAP-REDUCE):")
print("=" * 50)
print(map_reduce_summary)

### Tùy chỉnh prompts cho Map-Reduce

In [ ]:
# Custom prompt cho giai đoạn Map
map_prompt = PromptTemplate(
    template="""
Hãy tóm tắt phần văn bản sau thành 2-3 câu ngắn gọn, 
tập trung vào thông tin quan trọng nhất:

{text}

TÓM TẮT NGẮN GỌN:""",
    input_variables=["text"]
)

# Custom prompt cho giai đoạn Reduce  
reduce_prompt = PromptTemplate(
    template="""
Dưới đây là các tóm tắt từ các phần khác nhau của một văn bản dài:

{text}

Hãy kết hợp các tóm tắt này thành một tóm tắt tổng thể có cấu trúc rõ ràng, 
bao gồm:
- Giới thiệu chủ đề chính
- Các điểm quan trọng
- Kết luận

TÓM TẮT TỔNG THỆ:""",
    input_variables=["text"]
)

# Tạo custom Map-Reduce chain
custom_map_reduce_chain = load_summarize_chain(
    llm=llm,
    chain_type="map_reduce",
    map_prompt=map_prompt,
    combine_prompt=reduce_prompt,
    verbose=True
)

print("🔄 Đang tóm tắt bằng custom Map-Reduce...")
custom_map_reduce_summary = custom_map_reduce_chain.run(split_docs)

print("\n📝 KẾT QUẢ TÓM TẮT (CUSTOM MAP-REDUCE):")
print("=" * 50)
print(custom_map_reduce_summary)

## 3. Phương pháp Refine

Phương pháp này xử lý từng phần văn bản một cách tuần tự, mỗi lần cải thiện và tinh chỉnh tóm tắt dựa trên phần mới.

In [ ]:
# Tạo Refine chain
refine_chain = load_summarize_chain(
    llm=llm,
    chain_type="refine",
    verbose=True
)

print("🔄 Đang tóm tắt bằng phương pháp Refine...")
refine_summary = refine_chain.run(split_docs)

print("\n📝 KẾT QUẢ TÓM TẮT (REFINE):")
print("=" * 50)
print(refine_summary)

### Tùy chỉnh prompts cho Refine

In [ ]:
# Custom prompt cho tóm tắt ban đầu
refine_initial_prompt = PromptTemplate(
    template="""
Hãy tóm tắt phần văn bản sau thành một đoạn ngắn gọn:

{text}

TÓM TẮT:""",
    input_variables=["text"]
)

# Custom prompt cho việc refine
refine_prompt = PromptTemplate(
    template="""
Bạn đã có một tóm tắt hiện tại:
{existing_answer}

Dưới đây là thêm thông tin từ văn bản:
{text}

Hãy cải thiện tóm tắt hiện tại bằng cách:
- Thêm thông tin quan trọng mới (nếu có)
- Loại bỏ thông tin không cần thiết
- Đảm bảo tính mạch lạc và logic

Nếu thông tin mới không quan trọng, giữ nguyên tóm tắt hiện tại.

TÓM TẮT CẢI THIỆN:""",
    input_variables=["existing_answer", "text"]
)

# Tạo custom Refine chain
custom_refine_chain = load_summarize_chain(
    llm=llm,
    chain_type="refine",
    question_prompt=refine_initial_prompt,
    refine_prompt=refine_prompt,
    verbose=True
)

print("🔄 Đang tóm tắt bằng custom Refine...")
custom_refine_summary = custom_refine_chain.run(split_docs)

print("\n📝 KẾT QUẢ TÓM TẮT (CUSTOM REFINE):")
print("=" * 50)
print(custom_refine_summary)

## So sánh các phương pháp

Hãy so sánh độ dài và chất lượng của các tóm tắt từ các phương pháp khác nhau.

In [ ]:
# So sánh độ dài các tóm tắt
summaries = {
    "Stuff": stuff_summary,
    "Custom Stuff": custom_stuff_summary,
    "Map-Reduce": map_reduce_summary,
    "Custom Map-Reduce": custom_map_reduce_summary,
    "Refine": refine_summary,
    "Custom Refine": custom_refine_summary
}

print("📊 SO SÁNH ĐỘ DÀI CÁC TÓM TẮT:")
print("=" * 60)
for method, summary in summaries.items():
    word_count = len(summary.split())
    char_count = len(summary)
    print(f"{method:<20}: {word_count:>4} từ, {char_count:>5} ký tự")

print(f"\nVăn bản gốc: {len(long_text.split()):>4} từ, {len(long_text):>5} ký tự")

## Tóm tắt văn bản từ URL

Ví dụ thực tế: tóm tắt nội dung từ một trang web.

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
import requests
from bs4 import BeautifulSoup

def load_web_content(url):
    """Load và extract text từ một URL"""
    try:
        # Tải nội dung web
        response = requests.get(url)
        response.raise_for_status()
        
        # Parse HTML
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Loại bỏ script và style tags
        for script in soup(["script", "style"]):
            script.extract()
        
        # Lấy text
        text = soup.get_text()
        
        # Làm sạch text
        lines = (line.strip() for line in text.splitlines())
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        text = ' '.join(chunk for chunk in chunks if chunk)
        
        return text
        
    except Exception as e:
        print(f"❌ Lỗi khi tải nội dung: {e}")
        return None

# Ví dụ với URL (thay thế bằng URL thực tế)
# url = "https://example.com/long-article"
# web_content = load_web_content(url)

# Sử dụng nội dung mẫu thay vì URL thực
web_content = """
Blockchain là một công nghệ cơ sở dữ liệu phân tán cho phép lưu trữ thông tin một cách an toàn và minh bạch.
Mỗi block trong blockchain chứa một số lượng giao dịch và được liên kết với block trước đó thông qua mã hash.
Điều này tạo ra một chuỗi các block không thể thay đổi, đảm bảo tính bảo mật và toàn vẹn dữ liệu.

Bitcoin, ra đời năm 2009, là ứng dụng đầu tiên và nổi tiếng nhất của blockchain.
Tuy nhiên, công nghệ blockchain có tiềm năng ứng dụng rộng rãi hơn nhiều so với chỉ là tiền điện tử.
Các lĩnh vực như supply chain, bất động sản, y tế, và voting đều có thể hưởng lợi từ blockchain.

Smart contracts là một tính năng quan trọng của blockchain thế hệ mới như Ethereum.
Những hợp đồng thông minh này có thể tự động thực thi khi các điều kiện được định trước được đáp ứng.
Điều này giúp giảm thiểu rủi ro và chi phí trong nhiều giao dịch kinh doanh.
"""

if web_content:
    print(f"✅ Đã tải nội dung: {len(web_content)} ký tự")
    
    # Tạo document từ web content
    web_docs = [Document(page_content=web_content)]
    
    # Tóm tắt nội dung web
    web_summary = stuff_chain.run(web_docs)
    
    print("\n📝 TÓM TẮT NỘI DUNG WEB:")
    print("=" * 50)
    print(web_summary)
else:
    print("❌ Không thể tải nội dung web")

## Tóm tắt có cấu trúc

Tạo tóm tắt theo định dạng cụ thể với các sections rõ ràng.

In [ ]:
# Prompt cho tóm tắt có cấu trúc
structured_prompt = PromptTemplate(
    template="""
Hãy tóm tắt văn bản sau theo cấu trúc:

## Chủ đề chính
[Mô tả ngắn gọn chủ đề chính của văn bản]

## Các điểm quan trọng
- [Điểm quan trọng 1]
- [Điểm quan trọng 2]
- [Điểm quan trọng 3]

## Kết luận
[Thông điệp hoặc kết luận chính]

Văn bản cần tóm tắt:
{text}

TÓM TẮT CÓ CẤU TRÚC:""",
    input_variables=["text"]
)

# Tạo structured summarization chain
structured_chain = load_summarize_chain(
    llm=llm,
    chain_type="stuff",
    prompt=structured_prompt,
    verbose=True
)

print("🔄 Đang tạo tóm tắt có cấu trúc...")
structured_summary = structured_chain.run(docs)

print("\n📝 TÓM TẮT CÓ CẤU TRÚC:")
print("=" * 50)
print(structured_summary)

## Tóm tắt với độ dài tùy chỉnh

Điều chỉnh độ dài tóm tắt theo yêu cầu cụ thể.

In [ ]:
def create_length_specific_summary(text, target_length="ngắn"):
    """Tạo tóm tắt với độ dài cụ thể"""
    
    length_instructions = {
        "rất ngắn": "trong 1-2 câu",
        "ngắn": "trong 3-4 câu",
        "trung bình": "trong 1 đoạn văn (5-7 câu)",
        "dài": "trong 2-3 đoạn văn"
    }
    
    instruction = length_instructions.get(target_length, "trong 3-4 câu")
    
    length_prompt = PromptTemplate(
        template=f"""
Hãy tóm tắt văn bản sau {instruction}:

{{text}}

TÓM TẮT ({target_length.upper()}):""",
        input_variables=["text"]
    )
    
    chain = load_summarize_chain(
        llm=llm,
        chain_type="stuff",
        prompt=length_prompt
    )
    
    return chain.run([Document(page_content=text)])

# Tạo tóm tắt với các độ dài khác nhau
lengths = ["rất ngắn", "ngắn", "trung bình", "dài"]

print("📏 TÓM TẮT VỚI CÁC ĐỘ DÀI KHÁC NHAU:")
print("=" * 60)

for length in lengths:
    summary = create_length_specific_summary(long_text, length)
    word_count = len(summary.split())
    
    print(f"\n🔸 TÓM TẮT {length.upper()} ({word_count} từ):")
    print("-" * 40)
    print(summary)
    print()

## Đánh giá chất lượng tóm tắt

Sử dụng Claude để đánh giá chất lượng các tóm tắt.

In [ ]:
def evaluate_summary_quality(original_text, summary, method_name):
    """Đánh giá chất lượng tóm tắt"""
    
    evaluation_prompt = f"""
Hãy đánh giá chất lượng của tóm tắt sau đây dựa trên các tiêu chí:
1. Độ chính xác (có bao gồm thông tin quan trọng không?)
2. Độ ngắn gọn (có loại bỏ được thông tin không cần thiết không?)
3. Tính mạch lạc (có dễ hiểu và logic không?)
4. Độ bao quát (có nắm bắt được toàn bộ nội dung không?)

Văn bản gốc:
{original_text[:1000]}...

Tóm tắt cần đánh giá:
{summary}

Hãy cho điểm từ 1-10 cho mỗi tiêu chí và giải thích ngắn gọn.
Định dạng: Tiêu chí: [Điểm]/10 - [Giải thích ngắn]
"""
    
    evaluation = llm.invoke(evaluation_prompt).content
    return evaluation

# Đánh giá một số tóm tắt
print("🎯 ĐÁNH GIÁ CHẤT LƯỢNG TÓM TẮT:")
print("=" * 60)

methods_to_evaluate = [
    ("Custom Stuff", custom_stuff_summary),
    ("Map-Reduce", map_reduce_summary),
    ("Refine", refine_summary)
]

for method, summary in methods_to_evaluate:
    print(f"\n🔍 ĐÁNH GIÁ PHƯƠNG PHÁP: {method}")
    print("-" * 40)
    
    evaluation = evaluate_summary_quality(long_text, summary, method)
    print(evaluation)
    print()

## Best Practices và Tips

### 1. Chọn phương pháp phù hợp

In [ ]:
def choose_summarization_method(text_length, quality_priority="balanced"):
    """Gợi ý phương pháp tóm tắt dựa trên độ dài văn bản và ưu tiên"""
    
    recommendations = {
        "short": {
            "speed": "stuff",
            "quality": "stuff", 
            "balanced": "stuff"
        },
        "medium": {
            "speed": "stuff",
            "quality": "refine",
            "balanced": "map_reduce"
        },
        "long": {
            "speed": "map_reduce",
            "quality": "refine",
            "balanced": "map_reduce"
        }
    }
    
    # Phân loại độ dài
    if text_length < 2000:
        size_category = "short"
    elif text_length < 10000:
        size_category = "medium"
    else:
        size_category = "long"
    
    recommended_method = recommendations[size_category][quality_priority]
    
    return {
        "size_category": size_category,
        "recommended_method": recommended_method,
        "reason": f"Với văn bản {size_category} và ưu tiên '{quality_priority}', phương pháp '{recommended_method}' là tối ưu."
    }

# Ví dụ sử dụng
text_length = len(long_text)
recommendation = choose_summarization_method(text_length, "quality")

print("💡 GỢI Ý PHƯƠNG PHÁP TÓM TẮT:")
print("=" * 50)
print(f"Độ dài văn bản: {text_length} ký tự")
print(f"Phân loại: {recommendation['size_category']}")
print(f"Phương pháp đề xuất: {recommendation['recommended_method']}")
print(f"Lý do: {recommendation['reason']}")

### 2. Xử lý lỗi và fallback

In [ ]:
def robust_summarize(text, preferred_method="stuff", max_retries=3):
    """Tóm tắt với xử lý lỗi và fallback"""
    
    methods = ["stuff", "map_reduce", "refine"]
    
    # Đưa preferred_method lên đầu
    if preferred_method in methods:
        methods.remove(preferred_method)
        methods.insert(0, preferred_method)
    
    docs = [Document(page_content=text)]
    
    # Nếu text quá dài, tự động split
    if len(text) > 8000:  # Ngưỡng an toàn cho context window
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=100
        )
        docs = text_splitter.create_documents([text])
        
        # Loại bỏ "stuff" khỏi danh sách nếu text quá dài
        if "stuff" in methods:
            methods.remove("stuff")
    
    for method in methods:
        for attempt in range(max_retries):
            try:
                print(f"🔄 Thử phương pháp '{method}' (lần {attempt + 1})...")
                
                chain = load_summarize_chain(
                    llm=llm,
                    chain_type=method,
                    verbose=False
                )
                
                summary = chain.run(docs)
                
                print(f"✅ Thành công với phương pháp '{method}'")
                return {
                    "summary": summary,
                    "method_used": method,
                    "attempts": attempt + 1
                }
                
            except Exception as e:
                print(f"❌ Lỗi với phương pháp '{method}': {str(e)}")
                if attempt < max_retries - 1:
                    print("🔄 Thử lại...")
                else:
                    print(f"❌ Đã thử hết {max_retries} lần với phương pháp '{method}'")
    
    return {
        "summary": "Không thể tóm tắt văn bản do gặp lỗi với tất cả phương pháp.",
        "method_used": None,
        "attempts": max_retries * len(methods)
    }

# Test robust summarization
print("🛡️ TEST TÓM TẮT AN TOÀN:")
print("=" * 50)

result = robust_summarize(long_text, preferred_method="map_reduce")

print(f"\n📝 KẾT QUẢ:")
print(f"Phương pháp sử dụng: {result['method_used']}")
print(f"Số lần thử: {result['attempts']}")
print(f"\nTóm tắt:\n{result['summary']}")

## Kết luận

### Tóm tắt các phương pháp:

| Phương pháp | Ưu điểm | Nhược điểm | Khi nào sử dụng |
|-------------|---------|------------|------------------|
| **Stuff** | Nhanh, đơn giản, giữ nguyên context | Giới hạn độ dài văn bản | Văn bản ngắn, cần tốc độ |
| **Map-Reduce** | Xử lý văn bản dài, có thể song song | Có thể mất thông tin liên kết | Văn bản dài, cần tốc độ |
| **Refine** | Chất lượng cao, giữ nguyên ngữ cảnh | Chậm, xử lý tuần tự | Cần chất lượng tóm tắt tốt nhất |

### Best Practices:

1. **Chọn phương pháp phù hợp** dựa trên độ dài văn bản và yêu cầu chất lượng
2. **Sử dụng custom prompts** để có tóm tắt theo định dạng mong muốn
3. **Xử lý lỗi** và có fallback methods
4. **Kiểm tra độ dài** văn bản trước khi chọn phương pháp
5. **Đánh giá chất lượng** tóm tắt để cải thiện

### Tips cho production:

- Cache các tóm tắt để tránh gọi API không cần thiết
- Sử dụng streaming để cải thiện user experience
- Monitor token usage để tối ưu chi phí
- Implement rate limiting để tránh vượt quota
- A/B test các phương pháp để tìm ra phù hợp nhất